# InCal

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
from pandas.api.types import CategoricalDtype
from IPython.display import display, HTML
from statsmodels.formula.api import ols
from collections import OrderedDict, Counter
from jupyter_dash import JupyterDash 
import dash_html_components as html
from dash import dcc
from dash import no_update
from dash import dash_table
import dash
from dash.dependencies import Input, Output

# Step 1 - **Upload data**

run cell to upload the data


---



---



In [3]:
dataframes = {}
is_one_file = False
# 'this code block for the jupter colab only'
# try:
#     from google.colab import files
#     import io
#     uploaded = files.upload()
#     if len(uploaded) > 1:
#         for fn in uploaded.keys():
#             print('User uploaded file "{name}" with length {length} bytes'.format(
#                 name=fn, length=len(uploaded[fn])))
#             dataframes[fn] = pd.read_csv(io.BytesIO(uploaded[fn]))
#             is_one_file = False
#     else:
#         name = list(uploaded.keys())[0]
#         dataframes = pd.read_csv(io.BytesIO(uploaded[name]))
# except:
#     print('check the if error - file.csv, there more then one table in the sheet?, is table has missing columns?')

# dataframes = {
#     '3': pd.read_csv("csvs/shani_exp/hebrew_2021-07-28_16_33_hebrew16_shani_w2_acdoors_pt1_m_calr.csv", parse_dates=['Date_Time_1']),
#     '4': pd.read_csv("csvs/shani_exp/hebrew_2021-08-01_13_49_hebrew16_shani_w1_pt2b_m_calr(1).csv", parse_dates=['Date_Time_1']),
#     '5': pd.read_csv("csvs/shani_exp/hebrew_2021-08-04_11_45_hebrew16_shani_acdoors_w2p1_m_calr.csv", parse_dates=['Date_Time_1']),
#     '6': pd.read_csv("csvs/all_weeks/hebrew_2021-08-10_16_15_hebrew16_shani_w2p2.1_m_calr.csv", parse_dates=['Date_Time_1']),
#     '7': pd.read_csv("csvs/all_weeks/hebrew_2021-08-11_16_24_hebrew16_shani_acdoors_w3_m_calr.csv", parse_dates=['Date_Time_1']),
#     '8': pd.read_csv("csvs/all_weeks/hebrew_2021-08-15_16_24_hebrew16_sahni_acdoors_w3p2_m_calr.csv", parse_dates=['Date_Time_1']),
#     '9': pd.read_csv("csvs/all_weeks/hebrew_2021-08-19_16_17_hebrew16_shani_acdoors_w4_m_calr.csv", parse_dates=['Date_Time_1']),
#     '10': pd.read_csv("csvs/all_weeks/hebrew_2021-08-26_16_12_hebrew16_shani_acdoors_w5_m_calr.csv", parse_dates=['Date_Time_1']),
#     '11': pd.read_csv("csvs/all_weeks/hebrew_2021-08-29_08_41_hebrew16_shani_acdoors_w5_dd_m_calr.csv", parse_dates=['Date_Time_1']),
#     '12': pd.read_csv("csvs/all_weeks/hebrew_2021-09-02_07_54_hebrew16_dark dark week2_m_calr.csv", parse_dates=['Date_Time_1'])
# }
# dataframes = pd.read_csv('csvs\shani_exp\hebrew_2021-07-28_16_33_hebrew16_shani_w2_acdoors_pt1_m_calr.csv', parse_dates=['Date_Time_1'])
dataframes = pd.read_csv('example_csvs/examle.csv', parse_dates=['Date_Time_1'])
dataframes

{'3':             Date_Time_1  envirolightlux_2  envirotemp_2  envirorh_2  \
 0   2021-07-28 16:45:00                 0             0           0   
 1   2021-07-28 17:05:00                 0             0           0   
 2   2021-07-28 17:25:00                 0             0           0   
 3   2021-07-28 17:45:00                 0             0           0   
 4   2021-07-28 18:05:00                 0             0           0   
 ..                  ...               ...           ...         ...   
 273 2021-08-01 11:45:00                 0             0           0   
 274 2021-08-01 12:05:00                 0             0           0   
 275 2021-08-01 12:25:00                 0             0           0   
 276 2021-08-01 12:45:00                 0             0           0   
 277 2021-08-01 13:05:00                 0             0           0   
 
      envirooccupancy_2  envirosound_2     vo2_1     vo2_2     vo2_3     vo2_4  \
 0                    0              0  1.42134

# Step 2 - **INSERT EXPERIMENT DATA**

---



---



In [308]:
dict_groups = OrderedDict(Control = [1, 4, 7, 10, 13], Group_2 = [3, 5, 9, 12, 16], Group_3 = [2, 6, 8, 11, 14, 15])
print(f"{dict_groups}")

OrderedDict([('Control', [1, 4, 7, 10, 13]), ('Group_2', [3, 5, 9, 12, 16]), ('Group_3', [2, 6, 8, 11, 14, 15])])


## names replacement

...

In [309]:
name_for_replacement_in_data_table = {
    'bodymass': 'Weight_(gr)', 
    'vo2': 'Oxygen_Consumption_(ml/hr)', 
    'vco2': 'Carbon_Dioxide_Production_(ml/hr)',
    'kcal_hr': 'Energy_Expenditure_(kcal/hour)',
    'foodupa': 'Cumulative_Food_Intake_(kcal)',
    'rq': 'Respiratory_Exchange_Ratio',
    'pedmeters': 'Pedestrian_Locomotion_(m)',
    'allmeters': 'Total_Distance_includes_fine_movement_(m)',
    'waterupa': 'Cumulative_Water_Intake_(ml)',
    'Wheel': 'Total_Wheel_Counts_(Counts)' # need to check about the wheel parme...
}

calculeted_parmeters = {
    'water': 'Hourly_Water_Intake_(ml)',   
    'food': 'Hourly_Food_Intake_(kcal)',
    'locomotor_activity': 'Locomotor_Activity_(beam_breaks)',
    'energy_balance': 'Energy_Balance_(kcal/hour)',
}

# **Prepring the data**

## Functions for Prepring

-> run cell

In [310]:
def sort_data_by_ids(df, column_name):
    return df.sort_values(column_name)

def flat_list(d_list):
    '''
    dependencies: itertools
    '''
    return list(itertools.chain.from_iterable(d_list))

def slice_df_for_floats_and_category(df, column_name):
    return df.select_dtypes(include=['float64']), df.select_dtypes(include=['category'])

def get_subject_ids(df, column_name):
    return df[column_name].unique()

def calc_mean_and_std_for_df_by_ids(df, ids_column_name):
    return df.groupby(ids_column_name).agg([np.mean, np.std])

def get_lims_upper_and_lower(df_means_and_stds, number_of_ids, number_featuers_columns, by_sd_of=2):
    calcs_shape_values = df_means_and_stds.values.reshape(number_of_ids, number_featuers_columns, 2)
    means = calcs_shape_values[:, :, :1]
    stds = calcs_shape_values[:, :, 1:]
    upper_lims = means + stds * by_sd_of
    lower_lims = means - stds * by_sd_of
    return upper_lims, lower_lims

def reshpe_vlaues_3d_ndarray(ndarray, axis0_dimensions, axis1_columns, axis2_rows):
    return ndarray.reshape(axis0_dimensions, axis1_columns, axis2_rows)

def select_and_replace_outliers(ndarry_of_features, ndarry_uppers_lims, ndarry_lowers_lims):
    conditiones = [
        ndarry_of_features > ndarry_uppers_lims,
        ndarry_of_features < ndarry_lowers_lims
    ]
    choices = [np.nan, np.nan]
    return np.select(conditiones, choices, ndarry_of_features)
    
def back_to_2d_ndarray(ndarry_of_features, axis1, axis2):
    return ndarry_of_features.reshape(axis1, axis2)
    
def sort_data_by_index(df):
    return df.sort_index()

def get_categories_cals_names(df):
    return df.index.names[1:] 

def incal_get_categories_col_from_multiindex(df):
  levels_names = get_categories_cals_names(df)
  get_values_values_from_index = df.reset_index(level=levels_names)
  return get_values_values_from_index[levels_names]

def remove_outliers_mixed_df(df, column_ids_name):
    sorted_df = sort_data_by_ids(df, column_ids_name)
    fetuers, ids = slice_df_for_floats_and_category(sorted_df, column_ids_name)
    df_means_and_stds = calc_mean_and_std_for_df_by_ids(fetuers, ids[column_ids_name])
    number_of_ids = len(ids[column_ids_name].unique())
    fetuers_columns = fetuers.columns
    number_featuers_columns = len(fetuers_columns)
    upper_lims, lower_lims = get_lims_upper_and_lower(df_means_and_stds, number_of_ids, number_featuers_columns)
    dimensions_by_numbers_of_ids_upper_lims = reshpe_vlaues_3d_ndarray(upper_lims, number_of_ids, 1, number_featuers_columns) 
    dimensions_by_numbers_of_ids_lower_lims = reshpe_vlaues_3d_ndarray(lower_lims, number_of_ids, 1, number_featuers_columns)
    columns_of_each_id = fetuers.shape[0] // number_of_ids
    dimensions_by_numbers_of_ids_values = reshpe_vlaues_3d_ndarray(
        fetuers.values, 
        number_of_ids, 
        columns_of_each_id, 
        number_featuers_columns
    )
    outliers_replaced_to_nan_values_ndarray = select_and_replace_outliers(
        dimensions_by_numbers_of_ids_values, 
        dimensions_by_numbers_of_ids_upper_lims, 
        dimensions_by_numbers_of_ids_lower_lims
        )
    combien_axis0_and_axis1 = number_of_ids * columns_of_each_id
    original_df_shape = back_to_2d_ndarray(outliers_replaced_to_nan_values_ndarray, combien_axis0_and_axis1, number_featuers_columns)
    df_fetuers_without_outliers = pd.DataFrame(original_df_shape, columns=fetuers_columns, index=ids.index)
    df_without_outliers = pd.concat([ids, df_fetuers_without_outliers], axis=1)
    return sort_data_by_index(df_without_outliers)
# 17.1 ms ± 175 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)

def select_columns_by_metebolic_parm(df, param_name, exclude=False):
    if exclude == True:
        mask = ~df.columns.str.contains(pat=param_name)
        return df.loc[:, mask]
    mask = df.columns.str.contains(pat=param_name)
    return df.loc[:, mask]

def selecting_multi_column_by_part_of_name(df, list_pattern_parm):
    return df.filter(regex='|'.join(list_pattern_parm))

def multi_columns_by_metabolic_param(df, list_met_param, number):
    # https://stackoverflow.com/questions/21285380/find-column-whose-name-contains-a-specific-string
    columns_for_calc = df.columns[df.columns.astype("string").str.contains(pat="|".join(list_met_param))]
    df_calc = df[columns_for_calc].apply(lambda x: x * number)
    drop_old_columns = df.drop(columns_for_calc, axis=1)
    return pd.concat([drop_old_columns, df_calc], axis=1)

def loop_func_and_dfs(dfs, func, *args):
    return [func(df, *args) for df in dfs]

def _get_columns_names_list(df):
    return df.columns.values.tolist()

def _make_dict_to_replace_names(columns_names_list, pattern_addition_to_parms):
    leng = len(columns_names_list)
    return {columns_names_list[i]: pattern_addition_to_parms + columns_names_list[i] for i in range(leng)}

def _get_actuals_values(df):
    df_actuals_features_calculeted = df.diff()
    first_row_df_cumuletive = df.iloc[0:1]
    return df_actuals_features_calculeted.fillna(first_row_df_cumuletive)

def incal_get_actuals_from_cumuletive(df, columns_pattern, pattern_addition_to_parms):
    # get just the cumuletive columns from the original df
    df_cumuletive_culumns = select_columns_by_metebolic_parm(df, columns_pattern)
    # get the columns names of the cumuletive columns
    columns_names = _get_columns_names_list(df_cumuletive_culumns)
    # dict to replace names
    dict_new_names = _make_dict_to_replace_names(columns_names, pattern_addition_to_parms)
    # replace the columns names of the actuals culumns
    df_actuals_features = df_cumuletive_culumns.rename(columns=dict_new_names)
    df_actuals = _get_actuals_values(df_actuals_features)
    return pd.concat([df, df_actuals], axis=1).drop(columns_names, axis=1)

def incal_calc_cumuletive_values(df, columns_pattern):
    select_cols = df.columns.astype("string").str.contains(pat = columns_pattern)
    actuals = df.loc[:, select_cols]
    actuals_columns_names = actuals.columns.values.tolist()
    new_cols_names = [name.replace(columns_pattern, '') for name in actuals_columns_names]
    langth = len(actuals_columns_names)
    cumuletive = actuals.rename(columns={actuals_columns_names[i]: new_cols_names[i] for i in range(langth)}).cumsum()
    return pd.concat([df, cumuletive], axis=1)

def incal_set_multindex(df, list_of_multi_index):
  ids_indexed_df = df.reset_index()
  return ids_indexed_df.set_index(list_of_multi_index)
  
def incal_groupby_then_agg(df, list_to_groupby, agg_func):
    groupby = df.groupby(list_to_groupby)
    return groupby.agg(agg_func)

def incal_resample(df_unstacked_subjects, role_to_resmple_by, agg_func):
    # refactoring - > make it more genric function
  # https://stackoverflow.com/questions/15799162/resampling-within-a-pandas-multiindex
  return incal_groupby_then_agg(
                df_unstacked_subjects, 
                [
                    pd.Grouper(level='Date_Time_1', freq=role_to_resmple_by), 
                    pd.Grouper(level='subjectID')
                ], 
                agg_func)

def _multi_index_df_unstack(df_multi_indexed):
  return df_multi_indexed.unstack()

def _return_original_stacked_df(df_unstacked_subjects):
  return df_unstacked_subjects.stack().reset_index(level=1)

def incal_cumsum(df, list_of_multi_index, list_columns_names_to_cumsum):
  multi_indexed_df = incal_set_multindex(df, list_of_multi_index)
  unstacked_df = _multi_index_df_unstack(multi_indexed_df)
  cumsum_columns = unstacked_df[list_columns_names_to_cumsum].cumsum()
  cumsum_columns.columns = cumsum_columns.columns.map(lambda s: (s[0] + '_cumsum', s[1])) 
  concat_cumsum_columns = pd.concat([unstacked_df, cumsum_columns], axis=1)
  return _return_original_stacked_df(concat_cumsum_columns)

def _right_sepert_first_underscore(string):
    return tuple(string.rsplit("_", 1))

def _assemble_multi_index_axis_1_df(df, d_list, axis_1_names=["", ""]):
    # make a multi index 
    mul_i_columns = pd.MultiIndex.from_tuples(d_list, names=axis_1_names)
    # assemble new dataframe with multi index columns  
    return pd.DataFrame(df.values, index=df.index, columns=mul_i_columns)
    # then stack level 1 to the columns (level 1 -> subjects names e.g. 1 2 3...)

def incal_wide_to_long_df(wide_df, col_subj_name='subjectID'):
    cols_names = _get_columns_names_list(wide_df)
    # sepert feature name from cage number and put it in a tuple together ('allmeters', '1')
    l_micolumns  = [_right_sepert_first_underscore(col) for col in cols_names]
    multi_index_axis_1_df = _assemble_multi_index_axis_1_df(
        wide_df, 
        l_micolumns, 
        ['', col_subj_name]
    )
    # https://pandas.pydata.org/docs/user_guide/reshaping.html
    return multi_index_axis_1_df.stack(level=1)

def flatten(lst_in_lst):
    lst = []
    for l in lst_in_lst:
        if type(l) in [list,tuple,set]:
            lst.extend(l)
        else:
            return lst_in_lst
    return lst

def order_categoreis_columns(df, **kargs):
    '''
    order_categoreis_columns make sure the group and subjects in the right order. This is for,
    the statiscal analysis. The groups and the subjects needs to be in order of the expriment design.
    In order the anova, ancova and anova with interaction to work properly
    
    '''
    for col_name, order in kargs.items():
        df[col_name] = pd.Categorical(df[col_name], ordered=True, categories=flatten(order))

def incal_create_groups_column(df, groups_dict, ID_column_name='subjectID', Group_column_name='Group'):
    # improving this code to be more ganeral 
    categoris = {
        ID_column_name: groups_dict.values(),
        Group_column_name: groups_dict.keys()
    }
    df.loc[0:, Group_column_name] = df[ID_column_name].astype('int64')
    df.loc[0:, ID_column_name] = df[ID_column_name].astype('int64')
    for k, v in groups_dict.items():
        df.loc[0:, Group_column_name] = df[Group_column_name].replace(v, k)
    for category in categoris:
        df.loc[:, category] = df[category]
        
def day_and_night(df, datetime_column='Date_Time_1', start=7, end=19):
    df = df.assign(
        time=lambda x: np.where(
          df[datetime_column].dt.hour.ge(start) 
          & df[datetime_column].dt.hour.lt(end), 'Day', 'Night')).dropna()
    return df

def incal_make_averages_table(df, columns_names_too_groupby=['Group', 'subjectID'], column_name_for_time_of_day='time'):
    full_day = df.groupby(by=columns_names_too_groupby, sort=True, dropna=True).mean().reset_index().dropna()
    full_day[column_name_for_time_of_day] = 'Full day'
    D_and_N_df = day_and_night(df).groupby(by=[column_name_for_time_of_day, *columns_names_too_groupby], sort=True, dropna=True).mean().reset_index().dropna()
    return pd.concat([full_day, D_and_N_df])

# day and night time this data use for the graph below
def make_lists_start_and_end_to_day_night_time(df, datetime64_column='Date_Time_1', start=7, end=19):
    array_data_list = df[datetime64_column].unique()
    Series_datetime64 = pd.Series(array_data_list, name=datetime64_column)
    mask_daylight = Series_datetime64.dt.hour.ge(start) & Series_datetime64.dt.hour.lt(end)
    start_end = []
    still_True = False
    for i in range(len(Series_datetime64)):
        if still_True and mask_daylight.iloc[i]:
            start_end.append(Series_datetime64.iloc[i])
            still_True = False
        elif not still_True and not mask_daylight.iloc[i]:
            start_end.append(Series_datetime64.iloc[i])
            still_True = True
    return start_end

# stats 
anova_features = [
  'rq', 
  'locomotor_activity', 
  'actual_pedmeters_cumsum', 
  'actual_allmeters_cumsum'
]
ancova_and_anova_with_interaction_features = [
  'Energy_Balance', 
  'kcal_hr',
  'vo2', 
  'vco2', 
  'actual_foodupa', 
  'actual_waterupa', 
  ]

def reanem_df_by_with_list_by_index(df, indexed_new_names):
  columns_names = df.columns.values.tolist()
  new_columns_names = indexed_new_names
  zip_lists = zip(columns_names, new_columns_names)
  dict_renamed_columns = {column_name: new_column_name for column_name, new_column_name in zip_lists}
  return df.rename(columns=dict_renamed_columns)

def concat_dfs(list_of_series_dfs):
  return pd.concat(list_of_series_dfs, axis=1)

def anova_with_interaction(df, metabolic_var, independent, categorical):
    return ols(f'{metabolic_var} ~ {independent} + C({categorical}) + {independent}:C({categorical})', data=df).fit().pvalues
def ancova(df, metabolic_var, independent, categorical):
    return ols(f'{metabolic_var} ~ {independent} + C({categorical})', data=df).fit().pvalues
def anova(df, metabolic_var, categorical):
    return ols(f'{metabolic_var} ~ C({categorical})', data=df).fit().pvalues

def make_pvalues_of_anova_analysis(df, m_vars, cat_var):
  return [anova(df, m_var, cat_var) for m_var in m_vars]
def make_pvalues_of_ancova_analysis(df, m_vars, independent, cat_var):
  return [ancova(df, m_var, independent, cat_var) for m_var in m_vars]
def make_pvalues_of_anova_with_interaction_analysis(df, m_vars, independent, cat_var):
  return [anova_with_interaction(df, m_var, independent, cat_var) for m_var in m_vars]

def match_case(case, df, list_of_features, independent, category_col_name):
  cases = {
    'anova': make_pvalues_of_anova_analysis(
                                df, 
                                list_of_features, 
                                category_col_name
                                ),
    'ancova': make_pvalues_of_ancova_analysis(
                    df, 
                    list_of_features, 
                    independent,
                    category_col_name
                    ), 
    'anova_with_interaction': make_pvalues_of_anova_with_interaction_analysis(
                    df, 
                    list_of_features, 
                    independent,
                    category_col_name
                    ), 
    
  }
  return cases[case]

def incal_create_pvalues_datafram(case, df, list_of_features, independent, category_col_name):
  results_from_anovafunction = match_case(case, df, list_of_features, independent, category_col_name)
  pvalues_dfs_concated = concat_dfs(results_from_anovafunction)
  return reanem_df_by_with_list_by_index(pvalues_dfs_concated, list_of_features)

def create_anovas_table(df):
    anova_df = incal_create_pvalues_datafram(
    'anova', 
    df, 
    anova_features, 
    'bodymass',
    'Group'
    )
    anova_with_interaction_df = incal_create_pvalues_datafram(
    'anova_with_interaction', 
    df, 
    ancova_and_anova_with_interaction_features, 
    'bodymass',
    'Group'
    )
    # algoritem that get each non p value in anova with interaction and replace it with anova values and fill nan where is needed
    ancova_df = incal_create_pvalues_datafram(
    'ancova', 
    df, 
    ancova_and_anova_with_interaction_features, 
    'bodymass',
    'Group'
    )
    return concat_dfs([anova_df, anova_with_interaction_df, ancova_df]).T

## Insert experimental design

In [311]:
unwanted_column = "|".join(['envirolightlux', 'envirooccupancy', 'envirorh', 'envirosound', 'envirotemp'])
cumulative_parm = "|".join(['food', 'water', 'allmeters', 'wheelmeters', 'pedmeters'])
not_for_use_columns = ['vh2o', 'xbreak', 'ybreak', 'index'] # do nothig with it becouse it not importent to delete now this columns
pattern_addition_to_parms = 'actual_'
regx_pattern_for_mean = 'vo2|vco2|vh2o|rq_|bodymass|rq|kcal_hr|break_'
regx_pattern_for_sum = 'water|food'

dict_aggrageted_function_for_column = {
  'Energy_Balance': 'mean',
  'actual_allmeters': 'mean',
  'actual_pedmeters': 'mean',
  'bodymass': 'mean',
  'kcal_hr': 'mean',
  'locomotor_activity': 'mean',
  'rq': 'mean',
  'vco2': 'mean',
  'vo2': 'mean',
  'vh2o': 'mean',
  'xbreak': 'mean',
  'ybreak': 'mean',
  'actual_foodupa': 'sum', 
  'actual_waterupa': 'sum',
}
regx_pattern_no_mean_or_sum = regx_pattern_for_sum + regx_pattern_for_mean

## **Make analysis table**

In [312]:
# Merge files!!
df_or_dfs_in_list = [dataframes] if is_one_file else dataframes.values()
dfs = [incal_get_actuals_from_cumuletive(df, cumulative_parm, pattern_addition_to_parms) for df in df_or_dfs_in_list]
dfs_concated = pd.concat(dfs)

dfs_concated_cleaned = select_columns_by_metebolic_parm(dfs_concated, unwanted_column, True) 

dfs_concated = incal_set_multindex(dfs_concated, ['Date_Time_1']).drop(columns='index')
analysis_format = incal_wide_to_long_df(dfs_concated)

analysis_format[['vco2', 'vh2o', 'vo2']] = analysis_format[['vco2', 'vh2o', 'vo2']].mul(60)
analysis_format[['actual_foodupa']] = analysis_format[['actual_foodupa']].mul(3.56)
analysis_format['Energy_Balance'] = analysis_format['actual_foodupa'].values - analysis_format['kcal_hr'].values
analysis_format['locomotor_activity'] = analysis_format[['xbreak', 'ybreak']].sum(axis=1)

resampled_analysis_format = incal_resample(analysis_format, 'H', dict_aggrageted_function_for_column)

featuers_to_cumsum_by = ['actual_pedmeters', 'actual_allmeters']
analysis_format = incal_cumsum(analysis_format, ['Date_Time_1', 'subjectID'], featuers_to_cumsum_by)
resampled_analysis_format = incal_cumsum(resampled_analysis_format, ['Date_Time_1', 'subjectID'], featuers_to_cumsum_by)

add_feature_for_agg = {
  **dict_aggrageted_function_for_column, 
  'actual_allmeters_cumsum': 'mean',
  'actual_pedmeters_cumsum': 'mean'
}

# analysis format - with original datetime samples
incal_create_groups_column(resampled_analysis_format, dict_groups, 'subjectID')
resampled_analysis_format = incal_set_multindex(resampled_analysis_format, ['Date_Time_1', 'subjectID', 'Group']) 
# analysis format - datetime agg hor each rol
incal_create_groups_column(resampled_analysis_format.reset_index(level=1), dict_groups, 'subjectID')
resampled_analysis_format = incal_set_multindex(resampled_analysis_format, ['Date_Time_1', 'subjectID', 'Group']) 
# analysis format - averages for each subject
analysis_format_indexed = incal_set_multindex(resampled_analysis_format, ['Date_Time_1', 'subjectID', 'Group'])
grouped_analysis_format_df = analysis_format_indexed.groupby(level=['subjectID', 'Group'])
analysis_format_calculeted = grouped_analysis_format_df.agg(add_feature_for_agg).dropna()
# TODO: need to understend why it not average 
analysis_format_calculeted['Energy_Balance'] = analysis_format_calculeted['actual_foodupa'].values - analysis_format_calculeted['kcal_hr'].values

display(analysis_format.head(2), analysis_format.shape)
display(resampled_analysis_format.head(2), resampled_analysis_format.shape)
display(analysis_format_calculeted.head(2), analysis_format_calculeted.shape)

,subjectID,Energy_Balance,actual_allmeters,actual_allmeters_cumsum,actual_foodupa,actual_pedmeters,actual_pedmeters_cumsum,actual_waterupa,bodymass,envirolightlux,...,envirosound,envirotemp,kcal_hr,locomotor_activity,rq,vco2,vh2o,vo2,xbreak,ybreak
Date_Time_1,,,,,,,,,,,,,,,,,,,,,
2021-07-28 16:45:00,1,-0.407835,0.820163,0.820163,0.000000,0.259127,0.259127,0.0,24.46564,NaN,...,NaN,NaN,0.407835,201.0,0.758679,64.8669,8.167320,85.28094,79.0,122.0
2021-07-28 16:45:00,10,-0.088056,1.404414,1.404414,0.313275,1.140263,1.140263,0.0,22.61497,NaN,...,NaN,NaN,0.401331,211.0,0.818524,67.9386,9.748566,82.76856,64.0,147.0


(46672, 22)

Energy_Balance  actual_allmeters  \
Date_Time_1         subjectID Group                                       
2021-07-28 16:00:00 1         Control       -0.407835          0.820163   
                    10        Control       -0.088056          1.404414   

                                       actual_allmeters_cumsum  \
Date_Time_1         subjectID Group                              
2021-07-28 16:00:00 1         Control                 0.820163   
                    10        Control                 1.404414   

                                       actual_foodupa  actual_pedmeters  \
Date_Time_1         subjectID Group                                       
2021-07-28 16:00:00 1         Control        0.000000          0.259127   
                    10        Control        0.313275          1.140263   

                                       actual_pedmeters_cumsum  \
Date_Time_1         subjectID Group                              
2021-07-28 16:00:00 1         Control                 0.259127   
                    10        Control                 1.140263   

                                       actual_waterupa  bodymass   kcal_hr  \
Date_Time_1         subjectID Group                                          
2021-07-28 16:00:00 1         Control              0.0  24.46564  0.407835   
                    10        Control              0.0  22.61497  0.401331   

                                       locomotor_activity        rq     vco2  \
Date_Time_1         subjectID Group                                            
2021-07-28 16:00:00 1         Control               201.0  0.758679  64.8669   
                    10        Control               211.0  0.818524  67.9386   

                                           vh2o       vo2  xbreak  ybreak  
Date_Time_1         subjectID Group                                        
2021-07-28 16:00:00 1         Control  8.167320  85.28094    79.0   122.0  
                    10        Control  9.748566  82.76856    64.0   147.0

(15664, 16)

,,Energy_Balance,actual_allmeters,actual_pedmeters,bodymass,kcal_hr,locomotor_activity,rq,vco2,vo2,vh2o,xbreak,ybreak,actual_foodupa,actual_waterupa,actual_allmeters_cumsum,actual_pedmeters_cumsum
subjectID,Group,,,,,,,,,,,,,,,,
1,Control,534.113445,2.983936,2.315741,26.837848,0.423300,371.066054,0.823541,72.467427,87.072096,7.502909,166.129043,204.937011,534.536745,77.983681,1549.458299,1226.974891
2,Group_3,214.660840,1.943414,1.435875,25.403631,0.396033,237.896834,0.853766,69.852410,80.887200,9.081304,55.293667,182.603166,215.056873,89.802372,971.326403,710.293007


(16, 16)

# **Dashboard**

In [313]:
def assemble_dash_table(dff):
    return {
        'columns': [{"name": i, "id": i} for i in dff.columns],
        'data': dff.to_dict('records')
    }

def get_start_and_end_time(tuple_start_end, dict_time_stamps):
    start, end = (str(i) for i in tuple_start_end)
    return dict_time_stamps[start], dict_time_stamps[end]

def get_data(__global_df__, categories_columns_names):
    # need to find solution to the use of global var
    time_series = __global_df__.reset_index(level=categories_columns_names)
    order_categoreis_columns(time_series, subjectID=dict_groups.values(), Group=dict_groups.keys())
    return time_series

def trim_df_datetime(df, start_time, end_time):
    return time_series.loc[start_time:end_time]

def remove_data(df, outliers_true, start_time, end_time):
    if not outliers_true:
        return trim_df_datetime(df, start_time, end_time)
    outliers_removed = remove_outliers_mixed_df(df, 'subjectID')
    return trim_df_datetime(outliers_removed, start_time, end_time).dropna()

def get_ready_data_for_analysis(df, categories_columns_names, outliers_true, start_time, end_time):
    time_series = get_data(analysis_format_indexed, categories_columns_names).copy()
    return remove_data(time_series, outliers_true, start_time, end_time)

def assemble_data_to_dom(df, category_name, feature_name):
    features = [category_name, feature_name, 'bodymass'] if feature_name is 'bodymass' else [category_name, feature_name]
    return df.loc[:, features]

def get_group_or_individual_df(df, category_name, feature_agg):
    time_series_group_or_individual_dfs = {
        # change calculetion
        'Group': df.groupby([df.index, category_name]).agg(feature_agg).reset_index(1),
        'subjectID': df
        }
    return time_series_group_or_individual_dfs[category_name]

def graphs_maker(clean_df, df_grouped, group_or_individual_df_averages, averages_df, feature_name, category_name, colors):
    time_series_graph = px.scatter(
        df_grouped,
        x=df_grouped.index,
        y=feature_name,
        color=category_name, 
        color_discrete_sequence=colors, 
        template='simple_white'
    ).update_traces(mode='lines+markers')

    histogram = px.histogram(
        df_grouped, 
        x=feature_name, 
        color=category_name, 
        color_discrete_sequence=colors,
        template='simple_white'
        )

    box = px.box(
        df_grouped,
        x=category_name,
        y=feature_name,
        color=category_name,
        color_discrete_sequence=colors,
        template='simple_white'
        )
    
    averages = px.bar(
        group_or_individual_df_averages,
        x=category_name,
        y=feature_name,
        color=category_name,
        color_discrete_sequence=colors, 
        template='simple_white'
    )
    regression = px.scatter(
        averages_df,
        x=averages_df['bodymass'],
        y=feature_name,
        color='Group',
        color_discrete_sequence=colors, 
        template='simple_white', 
        trendline='ols'
    )  
    return time_series_graph, histogram, box, averages, regression

<>:31: SyntaxWarning:

"is" with a literal. Did you mean "=="?

<>:31: SyntaxWarning:

"is" with a literal. Did you mean "=="?

C:\Users\misha\AppData\Local\Temp/ipykernel_25272/3386431060.py:31: SyntaxWarning:

"is" with a literal. Did you mean "=="?



In [314]:
# STYLE =   {
#     'overflow': 'hidden',
#     'background-color': '#333',
#     'position': 'fixed',
#     'top': 0,
#     'width': '100%'
#   }

In [315]:
# dashboard code. 
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets = external_stylesheets)

time_stamps = analysis_format_indexed.index.get_level_values(0)
shape_analysis_format_indexed = analysis_format_indexed.shape
end_point_index_analysis_format_indexed = shape_analysis_format_indexed[0]-1
marks_indexed_time_stamp = {i: time_stamps[i] for i in range(shape_analysis_format_indexed[0])}

list_of_featuers = analysis_format_indexed.columns
categories_columns_names = get_categories_cals_names(analysis_format_indexed)
obj_categories_columns_names = [{'label': feature, 'value': feature} for feature in categories_columns_names]
obj_features = [{'label': feature, 'value': feature} for feature in list_of_featuers]

app.layout = html.Div(children=[
    dcc.Dropdown(
        id='show_as_group_or_individual',
        options=obj_categories_columns_names,
        value=categories_columns_names[0]
    ), 
    dcc.Dropdown(
        id='feature_exp',
        options=obj_features,
        value=list_of_featuers[1]
    ),
    dcc.Checklist(
    id='checklist_remove_outliers',
    options=[
        {'label': 'Remove outliers', 'value': 'True'},
    ],
    value=[],
    labelStyle={'display': 'inline-block'}
    ),
    dcc.RangeSlider(
        marks=marks_indexed_time_stamp,
        value=(0, end_point_index_analysis_format_indexed),
        id="range_slider_time_series", 
        allowCross=False,
        min=0,
        max=end_point_index_analysis_format_indexed,
    ),
    dcc.Graph(id = 'graph_time_series'), 
    dcc.Graph(id = 'box_graph'), 
    dcc.Graph(id = 'histogram_graph'), 
    dcc.Graph(id = 'graph_averages'), 
    dcc.Graph(id = 'graph_regression'), 
    dash_table.DataTable(id = 'p_values_table')
])

@app.callback(
    Output('graph_time_series', 'figure'), 
    Output('box_graph', 'figure'), 
    Output('histogram_graph', 'figure'), 
    Output('graph_averages', 'figure'), 
    Output('graph_regression', 'figure'), 
    Output('p_values_table', 'columns'), 
    Output('p_values_table', 'data'), 
    Input('feature_exp', 'value'),
    Input('show_as_group_or_individual', 'value'),
    Input('checklist_remove_outliers', 'value'),
    Input('range_slider_time_series', 'value'), 
    Input('range_slider_time_series', 'marks')
)
def poll_data(feature_name, category_name, outliers_true, tuple_start_end, dict_time_stamps):
    start_time, end_time = get_start_and_end_time(tuple_start_end, dict_time_stamps)
    # analsysis of expriment data - outliers and acclimetion
    clean_df = get_ready_data_for_analysis(
        analysis_format_indexed, 
        categories_columns_names, 
        outliers_true, 
        start_time, 
        end_time)
    # data to dom
    data_for_view = assemble_data_to_dom(clean_df, category_name, feature_name)
    # time series
    df_grouped = get_group_or_individual_df(data_for_view, category_name, add_feature_for_agg)
    # averages
    group_or_individual_df_averages = data_for_view.groupby([category_name]).mean().reset_index()
    # regression
    averages_df = clean_df.groupby(categories_columns_names).mean().dropna().reset_index()
    # make graphs
    colors = px.colors.qualitative.Vivid
    time_series_graph, histogram, box, averages, regression = graphs_maker(
        clean_df,
        df_grouped,
        group_or_individual_df_averages, 
        averages_df, 
        feature_name, 
        category_name, 
        colors
        )
    # need to refactore that remove the side affect of in this function 
    grouped_analysis_format_df = clean_df.groupby(categories_columns_names)
    analysis_format_calculeted = grouped_analysis_format_df.agg(add_feature_for_agg).dropna()
    analysis_format_calculeted['Energy_Balance'] = analysis_format_calculeted['actual_foodupa'].values - analysis_format_calculeted['kcal_hr'].values
    analysis_format_calculeted_index_reseted = analysis_format_calculeted.reset_index()
    p_values_table = create_anovas_table(analysis_format_calculeted_index_reseted)
    p_values_table = p_values_table.reset_index().rename(columns={'index':'Features'})
    columns = [{'id': p, 'name': p} for p in p_values_table.columns.to_list()]
    return time_series_graph, box, histogram, averages, regression, columns, p_values_table.to_dict('records')

In [316]:
def run_server(self,
               port=8004,
               debug=True,
               threaded=True,
               **flask_run_options):
    self.server.run(port=port, debug=debug, **flask_run_options)

if __name__ == '__main__':
    app.run_server(debug=True, port=8024, mode='external')

Dash app running on http://127.0.0.1:8024/


c:\Users\misha\Desktop\Research_Exercise\InCal\venv\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [317]:
# http://127.0.0.1:8024/

In [318]:
time_series = get_data(analysis_format_indexed, categories_columns_names).copy()
clean_df = remove_data(time_series, False, marks_indexed_time_stamp[0], marks_indexed_time_stamp[999])

In [319]:
clean_df

,subjectID,Group,Energy_Balance,actual_allmeters,actual_allmeters_cumsum,actual_foodupa,actual_pedmeters,actual_pedmeters_cumsum,actual_waterupa,bodymass,kcal_hr,locomotor_activity,rq,vco2,vh2o,vo2,xbreak,ybreak
Date_Time_1,,,,,,,,,,,,,,,,,,
2021-07-28 16:00:00,1,Control,-0.407835,0.820163,0.820163,0.000000,0.259127,0.259127,0.000000,24.465640,0.407835,201.0,0.758679,64.866900,8.167320,85.280940,79.000000,122.000000
2021-07-28 16:00:00,10,Control,-0.088056,1.404414,1.404414,0.313275,1.140263,1.140263,0.000000,22.614970,0.401331,211.0,0.818524,67.938600,9.748566,82.768560,64.000000,147.000000
2021-07-28 16:00:00,11,Group_3,-0.377329,3.553978,3.553978,0.006243,3.171110,3.171110,0.002538,21.349980,0.383572,360.0,0.794086,63.289920,8.856168,79.566900,150.000000,210.000000
2021-07-28 16:00:00,12,Group_2,-0.162136,11.915580,11.915580,0.204500,11.116600,11.116600,0.001420,24.696810,0.366636,788.0,0.843458,63.434640,7.688532,75.228840,191.000000,597.000000
2021-07-28 16:00:00,13,Control,-0.337903,3.425906,3.425906,0.116263,2.752747,2.752747,0.077571,25.822260,0.454166,440.0,0.772627,73.195680,8.843118,94.699680,11.000000,429.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-31 06:00:00,5,Group_2,-0.355115,0.777400,119.759237,0.000000,0.526800,84.927210,0.000000,23.124067,0.355115,46.0,0.790938,58.021070,10.948848,73.824820,4.666667,41.333333
2021-07-31 06:00:00,6,Group_3,-0.350968,0.162467,151.377481,0.000000,0.038667,105.699026,0.007518,26.114450,0.350968,31.0,0.938367,66.164620,10.064672,70.487080,12.333333,18.666667
2021-07-31 06:00:00,7,Control,-0.379044,0.811533,110.112684,0.000000,0.462033,77.980397,0.007445,24.657043,0.379044,126.0,0.844182,65.632766,6.982818,77.760440,56.666667,69.333333
